In [1]:
# WORKS ON EMPTY DATABASE - RUN ALL CELLS
# 3 SAMPLES ARE MISSING FROM METADATA BUT EXIST IN ALL_DATA - THEY HAVE BEEN DROPPED

import sqlite3
import pandas as pd

In [14]:
# Connect to the existing SQLite database
db_path = "./UrotheliomeData.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [15]:
# Enable foreign key constraints
conn.execute("PRAGMA foreign_keys = ON;")

In [16]:
# Create indexes for faster querying
# Rename to PascalCase
#cursor.execute("CREATE INDEX IF NOT EXISTS IdxGeneExpressionGeneName ON GeneExpression(GeneName);")
#cursor.execute("CREATE INDEX IF NOT EXISTS IdxSampleDatasetName ON Sample(DatasetName);")

In [129]:
metadata_file_path = "../JBU_data/metadata_v4.tsv"
all_data_file_path = "../JBU_data/all_data_test.tsv"

In [130]:
def load_metadata_tsv(file_path):
    df = pd.read_csv(file_path, sep='\t')
    return df

def load_all_data_tsv(file_path):
    df = pd.read_csv(file_path, sep='\t', low_memory=False)
    return df

def insert_into_db(df, table_name, conn):
    """Insert a DataFrame (clean) into the specified SQLite table."""
    try:
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"Successfully inserted data into {table_name}")
    except Exception as e:
        print(f"Error inserting into {table_name}: {e}")
        
def merge_all_datasets(base_path):
    ''' Merges all datasets from master.tsv and saves as all_data_test.tsv '''
    master_cv = pd.read_csv(base_path + "master.tsv", delimiter="\t")
    
    all_tsv = pd.DataFrame()
    for _, row in master_cv.iterrows():
        filename, _ = row["Filename"], row["Description"]
        df = pd.read_csv(base_path + filename, delimiter="\t")
        print("Dataset: {} Shape {}".format(filename, df.shape))

        # Exclude metadata file
        if "metadata" not in filename:
            if not all_tsv.empty:
                all_tsv = pd.merge(all_tsv, df, how="outer")
            else:
                all_tsv = df
    print('completed merging')
    return all_tsv

    # Uncomment to save as .tsv
    #all_tsv.to_csv(base_path + "all_data_test.tsv", sep="\t", index=False)

In [131]:
# currently equivalent to all_data.tsv but depends on master.tsv file contents
# merged_df = merge_all_datasets("../JBU_data/")

In [132]:
metadata_df = load_metadata_tsv(metadata_file_path)
all_data_df = load_all_data_tsv(all_data_file_path)

In [133]:
# Cleaning all_data_df
#print(all_data_df.info())
#print(all_data_df.dtypes)

# Convert all columns except 'gene' to numeric, setting errors='coerce' to convert invalid values to NaN
all_data_df.iloc[:, 1:] = all_data_df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
missing_values_count = all_data_df.iloc[:, 1:].isnull().sum().sum()
print(f"\nTotal NaN values found in expression data: {missing_values_count}")


Total NaN values found in expression data: 12884216


In [134]:
# Find rows where 'TCGA-ZF-AA56-01A' column has missing values
missing_rows = all_data_df[all_data_df['TCGA-ZF-AA56-01A'].isnull()]

# Find names of all genes (rows) with missing values
missing_genes = all_data_df[all_data_df.isnull().any(axis=1)]['genes'].tolist()

# Create a DataFrame of gene | number of samples with missing TPM value
missing_tpm_counts = all_data_df.isnull().sum(axis=1).reset_index()
missing_tpm_counts.columns = ['index', 'missing_count']
missing_tpm_counts = missing_tpm_counts[missing_tpm_counts['missing_count'] > 0]
missing_tpm_counts = missing_tpm_counts.merge(all_data_df[['genes']], left_on='index', right_index=True)
missing_tpm_counts = missing_tpm_counts[['genes', 'missing_count']]
missing_tpm_counts

,genes,missing_count
0,5S_rRNA,535
1,5_8S_rRNA,535
2,7SK,535
46,ABBA01000933.1,535
47,ABBA01000935.2,535
...,...,...
59161,ZNF454-DT,351
59263,ZNF573,535
59273,ZNF582-AS1,535
59311,ZNF622P1,351


In [135]:
#all_data_df[all_data_df['TCGA-ZF-AA56-01A'].isnull()]

In [136]:
categorical_columns = ["subset_name", "Dataset", "Tissue", "NHU_differentiation", 
                       "Substrate", "Gender", "tumor_stage", "vital_status"]
for col in categorical_columns:
    print(f"Unique values in {col}:")
    print(metadata_df[col].unique(), "\n")

Unique values in subset_name:
['B-AC_Undiff' 'B-AC_Diff' 'B-AC_P0' 'Ur-P0' 'CF-Undiff24H' 'CF-TZPD24H'
 'CF-Undiff144H' 'CF-TZPD144H' 'UW6H' 'SW6H' 'UW24H' 'SW24H' 'Cx32-EV'
 'Cx32-WT' 'Cx32-DN' 'Ur-Diff' 'Normoxia' 'Hypoxia' 'G9A-A366' 'G9A-A367'
 'G9A-A368' 'G9A-A369' 'G9A-UNC0646' 'G9A-UNC0647' 'G9A-UNC0648'
 'G9A-UNC0649' 'CON-18dpi' 'BKPyV-18dpi' 'BKPyV-IFNG-18dpi' 'IFNG-18dpi'
 'NR-3254' 'NR-3965' 'NR-742' 'NR-ABS' 'NR-BMS' 'NR-CON' 'NR-CP' 'NR-DEX'
 'NR-EB' 'NR-F12' 'NR-GC1' 'NR-ITE' 'NR-KSFM' 'NR-RZ' 'NR-TZ' 'NR2-BMS'
 'NR2-BMS-RZ' 'NR2-BMS-TZ' 'NR2-CON' 'NR2-KSFMc-1mMCa' 'NR2-LY295'
 'NR2-LY295-BMS' 'NR2-LY295-BMS-TZ' 'NR2-T007' 'NR2-T007-BMS'
 'NR2-T007-BMS-RZ' 'NR2-T007-BMS-TZ' 'NR2-T007-TZ' 'NR2-TZ' 'Scram48H'
 'HPGDkd48H' 'Scram144H' 'HPGDkd144H' 'LPCAT4kd48H' 'LPCAT4kd144H'
 'P0-cKIT-selection' 'P0-NGFR-selection' 'P0-pIgR-selection'
 'P0-after-selection' 'B-Diff' 'NR-DIF' 'Stroma-DMSO' 'Stroma-rSHH'
 'Uro-D' 'Uro-UD' 'Uro-P0' 'Stroma-DMSO24' 'Stroma-DMSO48' 'Stroma-SAG24

In [137]:
# Order of table creation + population
# - NHU
# - Dataset_Subset
# - Dataset
# - Tissue
# - Substrate
# - Gender
# - Tumor_Stage
# - Vital_Status
# - Gene
# - Sample
# - Gene_Expression

In [138]:
# NHU
nhu_df = metadata_df[['NHU_differentiation']].drop_duplicates().reset_index(drop=True)
nhu_df = nhu_df.dropna()
# Remove '?' row
nhu_df = nhu_df[nhu_df['NHU_differentiation'] != '?']
nhu_df = nhu_df.rename(columns={"NHU_differentiation": "NhuDifferentiation"})
nhu_df

,NhuDifferentiation
0,UD
1,ABS-Ca
2,P0
3,TZPD
5,NR


In [139]:
insert_into_db(nhu_df, "NHU", conn)

Error inserting into NHU: UNIQUE constraint failed: NHU.NhuDifferentiation


In [140]:
# Dataset_Subset
dataset_subset_df = metadata_df[['subset_name']].drop_duplicates().reset_index(drop=True)
dataset_subset_df = dataset_subset_df.rename(columns={"subset_name": "SubsetName"})
print(dataset_subset_df.to_string())

            SubsetName
0          B-AC_Undiff
1            B-AC_Diff
2              B-AC_P0
3                Ur-P0
4         CF-Undiff24H
5           CF-TZPD24H
6        CF-Undiff144H
7          CF-TZPD144H
8                 UW6H
9                 SW6H
10               UW24H
11               SW24H
12             Cx32-EV
13             Cx32-WT
14             Cx32-DN
15             Ur-Diff
16            Normoxia
17             Hypoxia
18            G9A-A366
19            G9A-A367
20            G9A-A368
21            G9A-A369
22         G9A-UNC0646
23         G9A-UNC0647
24         G9A-UNC0648
25         G9A-UNC0649
26           CON-18dpi
27         BKPyV-18dpi
28    BKPyV-IFNG-18dpi
29          IFNG-18dpi
30             NR-3254
31             NR-3965
32              NR-742
33              NR-ABS
34              NR-BMS
35              NR-CON
36               NR-CP
37              NR-DEX
38               NR-EB
39              NR-F12
40              NR-GC1
41              NR-ITE
42         

In [141]:
insert_into_db(dataset_subset_df, "DatasetSubset", conn)

Error inserting into DatasetSubset: UNIQUE constraint failed: DatasetSubset.SubsetName


In [142]:
# - Dataset
dataset_df = metadata_df[['Dataset']].drop_duplicates().reset_index(drop=True)
dataset_df = dataset_df.rename(columns={"Dataset": "DatasetName"})
dataset_df

,DatasetName
0,Baker_ABS-Ca
1,P0-Ureters
2,Fishwick_TZPD
3,ScratchWound
4,Cx32
5,Bladder-vs-Ureter
6,Hypoxia_G9A
7,Baker_BK
8,NuclearReceptor
9,LPCAT4kd_HPGDkd


In [143]:
insert_into_db(dataset_df, "Dataset", conn)

Error inserting into Dataset: UNIQUE constraint failed: Dataset.DatasetName


In [144]:
# - Tissue
tissue_df = metadata_df[['Tissue']].drop_duplicates().reset_index(drop=True)
tissue_df = tissue_df.rename(columns={"Tissue": "TissueName"})
tissue_df

,TissueName
0,Ureter
1,Bladder
2,CCL
3,MIBC


In [145]:
insert_into_db(tissue_df, "Tissue", conn)

Error inserting into Tissue: UNIQUE constraint failed: Tissue.TissueName


In [146]:
# - Substrate
substrate_df = metadata_df[['Substrate']].drop_duplicates().reset_index(drop=True)
# Remove nan
substrate_df = substrate_df.dropna()
# Remove '?' row
substrate_df = substrate_df[substrate_df['Substrate'] != '?']
substrate_df = substrate_df.rename(columns={"Substrate": "SubstrateType"})
substrate_df

,SubstrateType
0,P
1,M


In [147]:
insert_into_db(substrate_df, "Substrate", conn)

Error inserting into Substrate: UNIQUE constraint failed: Substrate.SubstrateType


In [148]:
# - Gender
gender_df = metadata_df[['Gender']].drop_duplicates().reset_index(drop=True)
# remove nan
gender_df = gender_df.dropna()
gender_df

,Gender
0,M
1,F


In [149]:
insert_into_db(gender_df, "Gender", conn)

Error inserting into Gender: UNIQUE constraint failed: Gender.Gender


In [150]:
# - Tumor_Stage
tumor_stage_df = metadata_df[['tumor_stage']].drop_duplicates().reset_index(drop=True)
tumor_stage_df = tumor_stage_df.dropna()
tumor_stage_df = tumor_stage_df.rename(columns={"tumor_stage": "Stage"})
tumor_stage_df

,Stage
1,Ta
2,T1
3,CIS
4,4
5,3
6,2
7,1


In [151]:
insert_into_db(tumor_stage_df, "TumorStage", conn)

Error inserting into TumorStage: UNIQUE constraint failed: TumorStage.Stage


In [152]:
# - Vital_Status
vital_status_df = metadata_df[['vital_status']].drop_duplicates().reset_index(drop=True)
vital_status_df = vital_status_df.dropna()
vital_status_df = vital_status_df.rename(columns={"vital_status": "Status"})
vital_status_df

,Status
1,dead
2,alive


In [153]:
insert_into_db(vital_status_df, "VitalStatus", conn)

Error inserting into VitalStatus: UNIQUE constraint failed: VitalStatus.Status


In [154]:
# - Gene
gene_df = all_data_df[['genes']].drop_duplicates().reset_index(drop=True)
gene_df = gene_df.rename(columns={"genes": "GeneName"})
gene_df

,GeneName
0,5S_rRNA
1,5_8S_rRNA
2,7SK
3,A1BG
4,A1BG-AS1
...,...
59649,ZZEF1
59650,ZZZ3
59651,hsa-mir-1253
59652,hsa-mir-423


In [155]:
insert_into_db(gene_df, "Gene", conn)

Error inserting into Gene: UNIQUE constraint failed: Gene.GeneName


In [156]:
# - Sample
metadata_df.replace({'?': None, 'NaN': None, '': None}, inplace=True)
metadata_df = metadata_df.where(pd.notna(metadata_df), None)  # Convert all NaNs to None

In [157]:
metadata_df.shape

(1294, 27)

In [158]:
metadata_df.columns

Index(['Sample', 'subset_name', 'shared_num_same_col', 'Dataset', 'Tissue',
       'NHU_differentiation', 'Substrate', 'TER', 'Gender', 'CCL_relatedness',
       'CCL_tissue_origin', 'Diagnosis', 'incl_ABS-Ca_diff_Bladder',
       'incl_ABS-Ca_diff_healthy_Bladder', 'incl_ABS-Ca_diff_Ureter',
       'incl_undiff_Bladder', 'incl_undiff_healthy_Bladder',
       'incl_undiff_Ureter', 'incl_P0_Bladder', 'incl_P0_healthy_Bladder',
       'incl_P0_Ureter', 'tumor_stage', 'vital_status', 'days_to_death',
       'TCGA408_classifier', 'MIBC_2019CC', 'MIBC_2019CC-noSR'],
      dtype='object')

In [159]:
#Remove unnecessary columns
metadata_df.drop(columns=["shared_num_same_col", "CCL_relatedness", "CCL_tissue_origin", "Diagnosis", 'incl_ABS-Ca_diff_Bladder',
       'incl_ABS-Ca_diff_healthy_Bladder', 'incl_ABS-Ca_diff_Ureter',
       'incl_undiff_Bladder', 'incl_undiff_healthy_Bladder',
       'incl_undiff_Ureter', 'incl_P0_Bladder', 'incl_P0_healthy_Bladder',
       'incl_P0_Ureter','TCGA408_classifier', 'MIBC_2019CC', 'MIBC_2019CC-noSR'], inplace=True)

In [160]:
#rename columns
metadata_df.rename(columns={"Sample": "SampleId", "subset_name": "SubsetName", "Dataset": "DatasetName", "Tissue": "TissueName", "Substrate": "SubstrateType", "Gender": "Gender", "tumor_stage": "Stage", "vital_status": "Status", "NHU_differentiation": "NhuDifferentiation", "TER": "TER", "days_to_death": "DaysToDeath"}, inplace=True)

In [161]:
# OLD METHOD OF INSERTING SAMPLE DATA
# for _, row in metadata_df.iterrows():
#     cursor.execute("""
#         INSERT INTO Sample (SampleId, SubsetName, DatasetName, TissueName, SubstrateType, Gender, 
#                             Stage, Status, NhuDifferentiation, TER, DaysToDeath)
#         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
#     """, (
#         row["Sample"], row["subset_name"], row["Dataset"], row["Tissue"], row["Substrate"],
#         row["Gender"], row["tumor_stage"], row["vital_status"], row["NHU_differentiation"],
#         row["TER"], row["days_to_death"]
#     ))

In [122]:
insert_into_db(metadata_df, "Sample", conn)

Successfully inserted data into Sample


In [121]:
metadata_df

,SampleId,SubsetName,DatasetName,TissueName,NhuDifferentiation,SubstrateType,TER,Gender,Stage,Status,DaysToDeath
0,Y1026_UD,B-AC_Undiff,Baker_ABS-Ca,Ureter,UD,P,None,M,None,None,NaN
1,Y1077_UD,B-AC_Undiff,Baker_ABS-Ca,Ureter,UD,P,None,M,None,None,NaN
2,Y1108_UD,B-AC_Undiff,Baker_ABS-Ca,Ureter,UD,P,None,M,None,None,NaN
3,Y1026_D,B-AC_Diff,Baker_ABS-Ca,Ureter,ABS-Ca,M,3922,M,None,None,NaN
4,Y1077_D,B-AC_Diff,Baker_ABS-Ca,Ureter,ABS-Ca,M,3526,M,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1289,TCGA-ZF-AA56-01A,BaSq,TCGA_MIBC,MIBC,None,None,None,F,3,dead,259.0
1290,TCGA-ZF-AA58-01A,BaSq,TCGA_MIBC,MIBC,None,None,None,F,4,alive,NaN
1291,TCGA-ZF-AA5H-01A,BaSq,TCGA_MIBC,MIBC,None,None,None,F,4,alive,NaN
1292,TCGA-ZF-AA5N-01A,LumP,TCGA_MIBC,MIBC,None,None,None,F,4,dead,168.0


In [162]:
# - Gene_Expression
all_data_df

# Drop columns if column name in {'TCGA-BL-A13I-01A', 'TCGA-BL-A13J-01A', 'TCGA-BL-A0C8-01A'}
columns_to_drop = {'TCGA-BL-A13I-01A', 'TCGA-BL-A13J-01A', 'TCGA-BL-A0C8-01A'}
all_data_df.drop(columns=columns_to_drop, inplace=True)

In [163]:
all_data_df

,genes,U0001,U0002,U0006,U0007,U0010,U0011,U0012,U0013,U0015,...,TCGA-ZF-AA4X-01A,TCGA-ZF-AA51-01A,TCGA-ZF-AA52-01A,TCGA-ZF-AA53-01A,TCGA-ZF-AA54-01A,TCGA-ZF-AA56-01A,TCGA-ZF-AA58-01A,TCGA-ZF-AA5H-01A,TCGA-ZF-AA5N-01A,TCGA-ZF-AA5P-01A
0,5S_rRNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,4.011284,0.000000,14.847300,2.713840,3.310313,2.645880,2.059660,0.000000,0.000000
1,5_8S_rRNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.058000,397.904000,1643.760000,697.995000,343.526000,510.180000,1215.172500,136.975000,1561.400000,264.411000
2,7SK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,91.984400,9.357257,15.890620,28.712000,0.211785,0.000000,0.180652,5.669090,350.865470,6.558230
3,A1BG,1.04,0.00,9.72,0.90,16.93,1.49,0.00,4.73,3.00,...,3.661039,8.442691,15.019774,51.167726,40.432533,5.951856,6.411895,2.223359,1.141739,4.435781
4,A1BG-AS1,0.35,5.71,8.79,0.35,5.45,0.00,0.12,1.81,1.86,...,0.880827,2.015884,2.210521,5.811502,6.002348,1.813384,2.542093,0.416332,0.298960,0.693852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59649,ZZEF1,15.11,22.48,19.44,17.08,36.61,23.15,15.19,16.38,37.73,...,12.586699,2.814479,3.243620,1.657250,7.187627,4.612398,4.061009,5.343059,3.477896,5.060469
59650,ZZZ3,43.14,86.31,75.87,49.57,74.94,60.92,75.92,69.92,79.70,...,13.410627,8.705693,8.893364,5.270915,17.553122,9.568289,9.072469,16.116666,5.063545,16.135274
59651,hsa-mir-1253,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
59652,hsa-mir-423,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [164]:
df_long = all_data_df.melt(id_vars=["genes"], var_name="sample_id", value_name="TPM")

In [165]:
df_long

,genes,sample_id,TPM
0,5S_rRNA,U0001,NaN
1,5_8S_rRNA,U0001,NaN
2,7SK,U0001,NaN
3,A1BG,U0001,1.040000
4,A1BG-AS1,U0001,0.350000
...,...,...,...
77192271,ZZEF1,TCGA-ZF-AA5P-01A,5.060469
77192272,ZZZ3,TCGA-ZF-AA5P-01A,16.135274
77192273,hsa-mir-1253,TCGA-ZF-AA5P-01A,0.000000
77192274,hsa-mir-423,TCGA-ZF-AA5P-01A,0.000000


In [166]:
df_long.dropna(subset=["TPM"], inplace=True)

In [167]:
df_long = df_long.rename(columns={"genes": "GeneName", "sample_id": "SampleId"})
df_long

,geneName,SampleId,TPM
3,A1BG,U0001,1.040000
4,A1BG-AS1,U0001,0.350000
5,A1CF,U0001,0.020000
6,A2M,U0001,132.120000
7,A2M-AS1,U0001,0.080000
...,...,...,...
77192271,ZZEF1,TCGA-ZF-AA5P-01A,5.060469
77192272,ZZZ3,TCGA-ZF-AA5P-01A,16.135274
77192273,hsa-mir-1253,TCGA-ZF-AA5P-01A,0.000000
77192274,hsa-mir-423,TCGA-ZF-AA5P-01A,0.000000


In [168]:
insert_into_db(df_long, "GeneExpression", conn)

Successfully inserted data into GeneExpression


In [43]:
# df_long = df_long[['sample_id', 'geneName', 'TPM']]
# df_long

,sample_id,geneName,TPM
3,U0001,A1BG,1.040000
4,U0001,A1BG-AS1,0.350000
5,U0001,A1CF,0.020000
6,U0001,A2M,132.120000
7,U0001,A2M-AS1,0.080000
...,...,...,...
77192271,TCGA-ZF-AA5P-01A,ZZEF1,5.060469
77192272,TCGA-ZF-AA5P-01A,ZZZ3,16.135274
77192273,TCGA-ZF-AA5P-01A,hsa-mir-1253,0.000000
77192274,TCGA-ZF-AA5P-01A,hsa-mir-423,0.000000


In [98]:
# Insert gene expression data
# for _, row in df_long.iterrows():
#     try:
#         cursor.execute("""
#             INSERT INTO Gene_Expression (sample_id, gene_name, TPM)
#             VALUES (?, ?, ?)
#         """, (row["sample_id"], row["gene_name"], row["TPM"]))
#     except sqlite3.IntegrityError:
#         # Skip if sample_id or gene_name does not exist
#         print(f"Skipping entry with missing sample or gene: {row['sample_id']}, {row['gene_name']}")

In [99]:
# Insert in bulk for efficiency
# data_tuples = list(df_long.itertuples(index=False, name=None))  # Convert DataFrame to list of tuples
# data_tuples

[('U0001', 'A1BG', 1.04),
 ('U0001', 'A1BG-AS1', 0.35),
 ('U0001', 'A1CF', 0.02),
 ('U0001', 'A2M', 132.12),
 ('U0001', 'A2M-AS1', 0.08),
 ('U0001', 'A2ML1', 0.0),
 ('U0001', 'A2ML1-AS1', 0.0),
 ('U0001', 'A2ML1-AS2', 0.0),
 ('U0001', 'A2MP1', 0.0),
 ('U0001', 'A3GALT2', 0.0),
 ('U0001', 'A4GALT', 50.04),
 ('U0001', 'A4GNT', 0.0),
 ('U0001', 'AA06', 0.0),
 ('U0001', 'AAAS', 31.11),
 ('U0001', 'AACS', 9.85),
 ('U0001', 'AACSP1', 0.0),
 ('U0001', 'AADAC', 0.0),
 ('U0001', 'AADACL2', 0.0),
 ('U0001', 'AADACL2-AS1', 0.0),
 ('U0001', 'AADACL3', 0.0),
 ('U0001', 'AADACL4', 0.0),
 ('U0001', 'AADACP1', 0.25),
 ('U0001', 'AADAT', 10.1),
 ('U0001', 'AAGAB', 94.86),
 ('U0001', 'AAK1', 17.89),
 ('U0001', 'AAMDC', 48.63),
 ('U0001', 'AAMP', 188.86),
 ('U0001', 'AANAT', 0.0),
 ('U0001', 'AAR2', 15.31),
 ('U0001', 'AARD', 0.0),
 ('U0001', 'AARS1', 28.43),
 ('U0001', 'AARS1P1', 0.0),
 ('U0001', 'AARS2', 10.52),
 ('U0001', 'AARSD1', 40.32),
 ('U0001', 'AARSD1P1', 0.18),
 ('U0001', 'AASDH', 8.95),
 ('U0

In [100]:
#len(data_tuples)

64308873

In [101]:
# try:
#     cursor.executemany("""
#         INSERT INTO Gene_Expression (sampleId, geneName, TPM)
#         VALUES (?, ?, ?)
#     """, data_tuples)
# except sqlite3.IntegrityError as e:
#     print("IntegrityError:", e)

In [11]:
# Save changes
conn.commit()

In [175]:
cursor.execute("SELECT COUNT(*) FROM GeneExpression")
row_count = cursor.fetchone()[0]
print(f"Total rows in GeneExpression: {row_count}")

Total rows in GeneExpression: 64308873


In [17]:
# Close without saving changes
conn.close()